In [5]:
!pip install spacy
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
%matplotlib inline

  Using cached https://files.pythonhosted.org/packages/9e/9b/62c60d2f5bc135d2aa1d8c8a86aaf84edb719a59c7f11a4316259e61a298/plac-0.9.6-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/b6/43/39372a0bc24d336dc88b87262c30f09d0a2c759f32a2965f90fb56da46f1/cymem-1.31.2-cp35-cp35m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/93/24/6ab1df969db228aed36a648a8959d1027099ce45fad67532b9673d533318/tqdm-4.23.4-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/2e/43/393e30e2768b0357541ac95891f96b80ccc4d517e0dd2fa3042fc8926538/msgpack_numpy-0.4.1-py2.py3-none-any.whl
  Found existing installation: cytoolz 0.9.0.1
    Uninstalling cytoolz-0.9.0.1:
      Successfully uninstalled cytoolz-0.9.0.1
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
!python -m spacy download en 

    100% |████████████████████████████████| 37.4MB 90.8MB/s ta 0:00:011    83% |██████████████████████████▉     | 31.3MB 86.3MB/s eta 0:00:01
  Running setup.py install for en-core-web-sm ... - \ | done
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful
    /usr/local/envs/py3env/lib/python3.5/site-packages/en_core_web_sm -->
    /usr/local/envs/py3env/lib/python3.5/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')



Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [7]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [8]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [9]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [92]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.at[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [93]:
import time
start_time = time.time()

# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)


print("--- %s seconds ---" % (time.time() - start_time))
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000
--- 3.604222536087036 seconds ---


,memorandum,deeply,convince,harden,shrill,sweet,print,past,twinkle,baby,...,cool,leg,discovery,distance,faint,ten,dinah'll,acquaintance,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


#### Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [94]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.9880877742946709

Test set score: 0.8895676691729323


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [169]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2002, 3066) (2002,)
Training set score: 0.9530469530469531

Test set score: 0.8862275449101796


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [96]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.8846394984326019

Test set score: 0.8735902255639098


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [97]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [98]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [99]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [100]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [101]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.6968680089485458


col_0,Austen,Carroll
row_0,,
Austen,1562,107
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

### Modeling with SVC

In [102]:
from sklearn.svm import SVC

svc = SVC()
train = svc.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', svc.score(X_train, y_train))
print('\nTest set score:', svc.score(X_test, y_test))


(3190, 3066) (3190,)
Training set score: 0.6824451410658308

Test set score: 0.6917293233082706


Using SVC didn't make a difference, in fact it was our worst performing model. Let's try making some new features from the suggestions above of sentence length, number of verbs, adverbs, nouns, and punctuation.

In [193]:
def parts_of_speech(df):
    counts = [] # initialize list
    for sentence in df.text_sentence: # loop through rows in df.text_sentence column, intialize vars to zero. 
        verbs = 0
        adverbs = 0
        nouns = 0
        puncuation = 0
        length_sentence = len(sentence)
        for token in sentence: #loop through words/tokens in sentence and add one if matched
            if token.pos_ == 'ADV':
                verbs +=1
            elif token.pos_ == 'VERB':
                adverbs +=1
            elif token.pos_ == 'NOUN':
                nouns +=1
            elif token.pos_ == 'PUNCT':
                puncuation +=1
        counts.append([length_sentence, verbs, adverbs, nouns, puncuation]) # append counts to each variable
    df_pos = pd.DataFrame.from_records(counts, columns=['verbs', 'adverbs', 'nouns','puncuation', 'length_sentence']) # turn lists into columns
    df_concat = pd.concat([df, df_pos], axis =1)
    
    return df_concat
wc_features = parts_of_speech(word_counts)

wc_features.head()
#boom

,memorandum,deeply,convince,harden,shrill,sweet,print,past,twinkle,baby,...,ten,dinah'll,acquaintance,text_sentence,text_source,verbs,adverbs,nouns,puncuation,length_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll,67,3,13,12,10
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll,63,7,11,8,7
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll,33,6,5,2,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Oh, dear, !)",Carroll,3,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(I, shall, be, late, !, ')",Carroll,6,0,2,0,2


And there we have our df with new features added. Let's train it and run it using logistic regression. 

In [105]:
#from above
Y1 = wc_features['text_source']
X1 = np.array(wc_features.drop(['text_sentence','text_source'], 1))

X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, 
                                                    Y1,
                                                    test_size=0.4,
                                                    random_state=0)

lr1 = LogisticRegression()
train = lr1.fit(X_train1, y_train1)
print(X_train1.shape, y_train1.shape)
print('Training set score:', lr1.score(X_train1, y_train1))
print('\nTest set score:', lr1.score(X_test1, y_test1))



(3190, 3071) (3190,)
Training set score: 0.9626959247648903

Test set score: 0.9210526315789473


All that work paid off to the tune of a .005 increase in accuracy. I'll take it. Moving on to the next part of the challenge.

"Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it."

In [107]:
 print(gutenberg.fileids()) 

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


Shakespeare would be too easy, so I'll go with Walt Whitman, but I'll clean a few others in case I decide to run them as well. 

In [139]:
# Load and clean the data.
leaves = gutenberg.raw('whitman-leaves.txt')
hamlet = gutenberg.raw('shakespeare-hamlet.txt')


# The Chapter indicator is idiosyncratic
leaves = re.sub(r'Chapter \d+', '', leaves)
hamlet = re.sub(r'CHAPTER .*', '', hamlet)


leaves = text_cleaner(leaves)
hamlet = text_cleaner(hamlet)



In [140]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
leaves_doc = nlp(leaves)
hamlet_doc = nlp(hamlet)


In [141]:
# Group into sentences.
leaves_sents = [[sent, "whitman"] for sent in leaves_doc.sents]
hamlet_sents = [[sent, "shakespeare"] for sent in hamlet_doc.sents]

# cut down lengths to match alice. 
leaves_sents = leaves_sents[0:len(alice_sents)]
hamlet_sents = hamlet_sents[0:len(alice_sents)]



In [142]:
# Build a new Bag of Words data frame for Persuasian word counts. 
# We'll use the same common words from Alice and Persuasion.
leaves_sentences = pd.DataFrame(leaves_sents)
leaves_bow = bow_features(leaves_sentences, common_words)


hamlet_sentences = pd.DataFrame(hamlet_sents)
hamlet_bow = bow_features(hamlet_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [189]:
'''# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    wc_features.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * wc_features.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)

#ValueError: all the input array dimensions except for the concatenation axis must match exactly'''

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [196]:
hamlet_bow_extra = parts_of_speech(hamlet_bow)

In [198]:
hamlet_bow_extra.head()

,memorandum,deeply,convince,harden,shrill,sweet,print,past,twinkle,baby,...,ten,dinah'll,acquaintance,text_sentence,text_source,verbs,adverbs,nouns,puncuation,length_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Actus, Primus, .)",shakespeare,3,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Scoena, Prima, .)",shakespeare,3,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Enter, Barnardo, and, Francisco, two, Centine...",shakespeare,7,0,1,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Barnardo, .)",shakespeare,2,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Who, 's, there, ?)",shakespeare,4,1,1,1,1


In [197]:
leaves_bow_extra = parts_of_speech(leaves_bow)
leaves_bow_extra.head()

,memorandum,deeply,convince,harden,shrill,sweet,print,past,twinkle,baby,...,ten,dinah'll,acquaintance,text_sentence,text_source,verbs,adverbs,nouns,puncuation,length_sentence
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Come, ,, said, my, soul, ,, Such, verses, for...",whitman,22,0,5,3,6
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(That, should, I, after, return, ,, Or, ,, lon...",whitman,41,3,2,9,12
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(Ever, with, pleas'd, smile, I, may, keep, on,...",whitman,20,5,3,2,3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,"(I, here, and, now, Signing, for, Soul, and, B...",whitman,19,2,2,1,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,(One's),whitman,1,0,0,0,0


In [204]:
#create variables
X_hamlet = hamlet_bow_extra.drop(['text_sentence','text_source'], 1)
y_hamlet = hamlet_bow_extra.text_source

#re-assign alice_wc and X_alice y_alice
alice = wc_features[wc_features.text_source == 'Carroll']
X_alice = alice.drop(['text_sentence','text_source'], 1)
y_alice = alice.text_source

#combine leaves dataset with Alice Data set
X_ha = pd.concat([X_hamlet, X_alice], 0)
y_ha = pd.concat([y_hamlet, y_alice], 0)

# train test split 60 train 40 test. 
X_train_ha, X_test_ha, y_train_ha, y_test_ha = train_test_split(X_ha, y_ha, test_size=0.4, random_state=0)

# Model.
lr2 = LogisticRegression()
train = lr2.fit(X_train_ha, y_train_ha)
print('Train set score:', lr2.score(X_train_ha, y_train_ha))
print('\nTest set score:', lr2.score(X_test_ha, y_test_ha))
lr2_predicted = lr2.predict(X_test_ha)
pd.crosstab(y_test_ha, lr2_predicted)

Train set score: 0.945054945054945

Test set score: 0.9019461077844312


col_0,Carroll,shakespeare
text_source,,
Carroll,543,104
shakespeare,27,662


In [205]:
#create variables
X_leaves = leaves_bow_extra.drop(['text_sentence','text_source'], 1)
y_leaves = leaves_bow_extra.text_source

#combine leves dataset with Alice Data set
X_la = pd.concat([X_leaves, X_alice], 0)
y_la = pd.concat([y_leaves, y_alice], 0)

# train test split 60 train 40 test. 
X_train_la, X_test_la, y_train_la, y_test_la = train_test_split(X_la, y_la, test_size=0.4, random_state=0)

# Model.
lr3 = LogisticRegression()
train = lr3.fit(X_train_la, y_train_la)
print('Train set score:', lr3.score(X_train_la, y_train_la))
print('\nTest set score:', lr3.score(X_test_la, y_test_la))
la_predicted = lr3.predict(X_test_la)
pd.crosstab(y_test_la, la_predicted)


Train set score: 0.965034965034965

Test set score: 0.9146706586826348


col_0,Carroll,whitman
text_source,,
Carroll,576,71
whitman,43,646


In [206]:
#create variables
#X_leaves = leaves_bow_extra.drop(['text_sentence','text_source'], 1)
#y_leaves = leaves_bow_extra.text_source

#re-assign paradise
paradise = wc_features[wc_features.text_source == 'Austen']
X_paradise = paradise.drop(['text_sentence','text_source'], 1)
y_paradise = paradise.text_source

#combine leves dataset with Alice Data set
X_lp = pd.concat([X_leaves, X_paradise], 0)
y_lp = pd.concat([y_leaves, y_paradise], 0)

# train test split 60 train 40 test. 
X_train_lp, X_test_lp, y_train_lp, y_test_lp = train_test_split(X_lp, y_lp, test_size=0.4, random_state=0)

# Model.
lr4 = LogisticRegression()
train = lr4.fit(X_train_lp, y_train_lp)
print('Train set score:', lr4.score(X_train_lp, y_train_lp))
print('\nTest set score:', lr4.score(X_test_lp, y_test_lp))
lr_lp_predicted = lr4.predict(X_test_lp)
pd.crosstab(y_test_lp, lr_lp_predicted)

Train set score: 0.95141065830721

Test set score: 0.8787593984962406


col_0,Austen,whitman
text_source,,
Austen,1347,125
whitman,133,523


In [207]:
#create variables
#X_hamlet = hamlet_bow.drop(['text_sentence','text_source'], 1)
#y_hamlet = hamlet_bow.text_source

#combine leves dataset with Alice Data set
X_hp = pd.concat([X_hamlet, X_paradise], 0)
y_hp = pd.concat([y_hamlet, y_paradise], 0)

# train test split 60 train 40 test. 
X_train_hp, X_test_hp, y_train_hp, y_test_hp = train_test_split(X_hp, y_hp, test_size=0.4, random_state=0)

# Model.
lr2 = LogisticRegression()
train = lr2.fit(X_train_hp, y_train_hp)
print('Train set score:', lr2.score(X_train_hp, y_train_hp))
print('\nTest set score:', lr2.score(X_test_hp, y_test_hp))
lr_hp_predicted = lr2.predict(X_test_hp)
pd.crosstab(y_test_hp, lr_hp_predicted)

Train set score: 0.9498432601880877

Test set score: 0.8853383458646616


col_0,Austen,shakespeare
text_source,,
Austen,1332,140
shakespeare,104,552


Our model performs over 90% when identiyfing sentences from Walt Whitman compared to either Austen or Carrol, but it falls just below 80% when identifying Shakespear from Austen or Carrol. I am some what suprised by this. I would have thought it would have been fairly easy to distinguish a shakespear sentence from that of anything on the list of texts. 